In [2]:
#!pip install azureml-contrib-fairness
#!pip install fairlearn==0.4.6
#!pip install raiwidgets
#pip install azureml-dataprep[pandas]

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Environment
from azureml.core import *
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.core import Workspace, Dataset, Datastore
import os, shutil
from azureml.core import Dataset
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.core import Workspace, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace,RunConfiguration
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep
from azureml.train.automl.utilities import get_primary_metrics
from azureml.data.datapath import DataPath
from azureml.core.datastore import Datastore
from azureml.pipeline.core import InputPortBinding
from azureml.core import Run
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.pipeline.core import PipelineParameter
from azureml.core import Dataset
from azureml.core import Environment
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.data import OutputFileDatasetConfig
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive import *
from azureml.widgets import RunDetails
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule
from azureml.core import Run
from azureml.core import get_run
import pandas as pd
import numpy as np

In [4]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- pip:
  - azureml-defaults
  - keras
  - tensorflow<=2.4.*
  - numpy
  - scikit-learn
  - pandas
  - matplotlib

Overwriting conda_dependencies.yml


In [6]:
subscription_id = '---------'
resource_group = '----'
workspace_name = 'interpretml'

# connection au WS
from azureml.core import Workspace
ws = Workspace.from_config()
ws.get_details()
myenv = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')


# Create a Python environment for the experiment
sklearn_env = Environment("sklearn-env")

# Ensure the required packages are installed
packages = CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults'])
sklearn_env.python.conda_dependencies = packages

# Create a script config + target compute ressource
script_config = ScriptRunConfig(source_directory="scripts",
                                script='training_script.py',
                                compute_target = 'jcharley3',
                                environment=myenv) 

# Submit the experiment
experiment = Experiment(workspace=ws, name='interpret2')
run = experiment.submit(config=script_config)
run.wait_for_completion()


{'runId': 'interpret2_1629749618_ec800433',
 'target': 'jcharley3',
 'status': 'Finalizing',
 'startTimeUtc': '2021-08-23T20:14:26.18289Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User program failed with SyntaxError: invalid syntax (training_script.py, line 24)',
   'messageParameters': {},
   'detailsUri': 'https://aka.ms/azureml-run-troubleshooting',
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'cf4d560c-d76a-464c-bd71-1c23f41f47ed',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json',
  'azureml.git.repository_uri': 'https://github.com/justine235/Azure_ML_Services.git',
  'mlflow.source.git.repoURL': 'https://github.com/justine235/Azure_ML_Services.git',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': 'bcebd4dcb2c40d4e56ca7018c7c7a4ae1f532d0a',
  'mlflow.source.

In [ ]:
#Récupération des fichiers de modèle
# List the files generated by the experiment
for file in run.get_file_names():
    print(file)

# Create a model folder in the current directory
os.makedirs('./output', exist_ok=True)

# Download the model from run history
run.register_model(model_name='model_save1', model_path='./output',description="Finetuning Similarity model")



TRACKING DES MODELES - LOCAL

In [ ]:
from azureml.core import Model
model = Model.register(workspace=ws,
                       model_name='rf',
                       model_path='output/model.pkl', # local path
                       description='rf test1',
                       tags={'data-format': 'CSV'},
                       model_framework=Model.Framework.SCIKITLEARN,
                       model_framework_version='0.20.3')

Afficher les modèles inscrits

In [ ]:
from azureml.core import Model

for model in Model.list(ws):
    # Get model name and auto-generated version
    print(model.name, 'version:', model.version)

ENREGISTREMENT D'UN JEU DE DONNEES

In [ ]:
from azureml.core import Dataset

blob_ds = ws.get_default_datastore()
csv_paths = [(blob_ds, 'data/train_dataset.csv')]
tab_ds = Dataset.Tabular.from_delimited_files(path=csv_paths)
tab_ds = tab_ds.register(workspace=ws, name='csv_table')

RUN UN MODELE ENREGISTRE

In [ ]:
run.register_model( model_name='classification_model',
                    model_path='outputs/model.pkl', # run outputs path
                    description='A classification model')

SCRIPT DE SCORING / INFERENCE TEMPS REELS
En règle générale, vous utilisez la fonction init pour charger le modèle à partir du registre de modèles et la fonction run pour générer des prédictions à partir des données d’entrée. L’exemple de script suivant illustre ce modèle :

In [ ]:
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the registered model file and load it
    model_path = Model.get_model_path('classification_model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Return the predictions as any JSON serializable format
    return predictions.tolist()

In [ ]:
# Créer l'environnement associé
from azureml.core.conda_dependencies import CondaDependencies

# Add the dependencies for your model
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

# Save the environment config as a .yml file
env_file = 'service_files/env.yml'
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

In [ ]:
#Une fois toute la configuration préparée, vous pouvez déployer le modèle
from azureml.core.model import Model

model = ws.models['classification_model']
service = Model.deploy(workspace=ws,
                       name = 'classifier-service',
                       models = [model],
                       inference_config = classifier_inference_config,
                       deployment_config = classifier_deploy_config,
                       deployment_target = production_cluster)
service.wait_for_deployment(show_output = True)

ESPACE DE RECHERCHE / HYPERPARAMETRE

In [ ]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn=0.24.1
- pip:
  - numpy==1.19.5
  - pandas==0.25.3
  - azureml-defaults

from azureml.core import Environment
sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

In [ ]:
#-------------------------- ETAPE 1 : MODEL INIT

# connection au WS
from azureml.core import Workspace
ws = Workspace.from_config()
ws.get_details()

# Execution du script en tant qu'experience
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal,RandomParameterSampling

# Create a Python environment for the experiment
#sklearn_env = Environment("sklearn-env")

# Ensure the required packages are installed
packages = CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults'])
sklearn_env.python.conda_dependencies = packages

# Create a script config + target compute ressource
script_config = ScriptRunConfig(source_directory=".",
                                script='training_script_hyperparametres.py',
                                compute_target = 'jcharley2',
                                environment=sklearn_env) 

# Submit the experiment
experiment = Experiment(workspace=ws, name='test_hyperparametres')
run  = experiment.submit(config=script_config)

from azureml.widgets import RunDetails
RunDetails(run).show()

In [ ]:
# save 
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = run.register_model(model_name='modele3', 
                           model_path='outputs/model.pkl',
                           model_framework=Model.Framework.SCIKITLEARN,
                           model_framework_version='0.24.1',
                           resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5))

# deploy
hosting_model = Model.deploy(ws, "modele3", [model])

In [ ]:
run.wait_for_completion(show_output=True)

In [ ]:
#-------------------------- ETAPE 2 : TUNNING PARAMETERS
# Espace de recherche des hyperparamètres
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice
    

param_sampling = RandomParameterSampling( {
    "--n_estimators": choice(15, 50, 100, 200, 300),
    "--criterion": choice("gini", "entropy")
    }
)


hyperdrive_config = HyperDriveConfig(run_config=script_config,
                                     hyperparameter_sampling=param_sampling, 
                                     primary_metric_name='accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=8,
                                     max_concurrent_runs=4)

# start the HyperDrive run
hyperdrive_run = experiment.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

In [ ]:
for child_run in run.get_children():
    print(child_run.id, child_run.get_metrics())

In [ ]:
#-------------------------- ETAPE 3 : ENREGISTREMENT

#récuperer le plus performant
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

In [ ]:
#host / save the best one 
model = best_run.register_model(model_name='rf_tuning', model_path='outputs/model_rf.joblib')